In [ ]:
SANDBOX_NAME = ''# Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"



# Spark ML Pipelines

Cargamos un dataset con información sobre cuán seguro es un coche. Con este dataset se estudiarán funciones muy importantes de Spark ML.



### Crear SparkSession

Sabemos que en Datio no es necesario crear la sesión de Spark ya al iniciar un notebook con el Kernel PySpark Python3 - Spark 2.1.0 se crea automáticamente. Pero así lo haríamos si fuera necesario.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()



### Cargar datos y comprobar schema

In [ ]:
cars = spark.read.csv(DATA_PATH+'data/automobile.csv', sep=';', header=True, inferSchema=True)

cars.printSchema()

In [ ]:
cars.show()



### Vamos a trabajar con los valores nulos



Una manera de devolver un dataframe sin filas que contengan nulos

In [ ]:
# Respuesta

cars.na.drop().show()



Si quisieramos reemplazar los valores nulos con otro valor:

In [ ]:
# Respuesta

cars.na.fill(50).show()



Otra forma de eliminar las filas con valores nulos (filtrar los nulos)

In [ ]:
# Respuesta

import pyspark.sql.functions as F

for column in cars.columns: 
    num_nulls = cars.where(F.col(column).isNull()).count()
    
    if num_nulls != 0:
        cars = cars.where(F.col(column).isNotNull())
        print("The column '{}' has {} nulls".format(column,num_nulls))
        print("The column '{}' has no more null values".format(column))



Cambiamos la variable objetivo para hacerla binaria, para poder utilizar algoritmos de clasificación binaria. 
* -2, -1, 0 => no es muy seguro (0)
* 1, 2, 3 => sí es seguro (1)

In [ ]:
# Respuesta

from pyspark.sql.types import DoubleType

cars = cars.withColumn('symboling_binary', F.udf(lambda value: 0. if value <= 0 else 1., DoubleType())(F.col('symboling')))



**Supongamos que queremos pasar la columna 'make' a dummy y luego lanzar un modelo de clasificación.**

**Resolviendo sin Pipeline** 

Tal como hemos visto antes, esto lo podríamos hacer paso a paso. 

1. Hacemos el StringIndexer a la columna _make_ para pasarla a numérica (0... n_categorias-1)
2. Hacemos el OneHotEncoder sobre el resultado del paso anterior para hacerla dummy
3. Seleccionamos las variables que vamos a incluir en nuestro modelo (todas aquellas que no sean string y que no sean la variable objetivo _symboling_). Eso lo hacemos recorriendo `df.dtypes`, el cual nos devuelve una lista de tuplas, donde cada tupla tiene (nombre_variable, tipo_variable).
4. Con las variables seleccionadas como predictoras del modelo, hacemos el VectorAssembler
5. Dividimos train y test
6. La salida del VectorAssembler será lo que le demos al modelo (en este caso un Random Forest). Entrenamos (*fit*) y predecimos (*transform*)

In [ ]:
# Respuesta

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

string_indexer = StringIndexer(inputCol='make', outputCol='make_indexed')
string_indexer_model = string_indexer.fit(cars)
cars_many_steps = string_indexer_model.transform(cars)

onehotencoder = OneHotEncoder(dropLast=False, inputCol= string_indexer.getOutputCol(), outputCol='make_encoded')
cars_many_steps = onehotencoder.transform(cars_many_steps)

columns_for_model = [element[0] for element in cars_many_steps.dtypes if element[1] != 'string' and 'symboling' not in element[0]]

vector_assembler = VectorAssembler(inputCols=columns_for_model, outputCol='assembled_features')
cars_many_steps = vector_assembler.transform(cars_many_steps)


cars_many_steps_train, cars_many_steps_test = cars_many_steps.randomSplit([0.8,0.2])

rf = RandomForestClassifier(featuresCol=vector_assembler.getOutputCol(), labelCol='symboling_binary')
rf_model = rf.fit(cars_many_steps_train)
cars_many_steps = rf_model.transform(cars_many_steps_test)

cars_many_steps.show()




**Resolviendo con Pipeline** 

Es parecido a lo que hicimos sin pipeline, pero en lugar de hacer:
- Crear el objeto string indexer, hacer *fit*, hacer *transform*
- Crear el objeto OHE, hacer *transform*
- Etc. 

Lo que hacemos es simplemente crear los objetos, los metemos como *stages* del pipeline, y luego le hacemos *fit* y *transform* al pipeline. Vamos a verlo.

In [ ]:
# Respuesta

from pyspark.ml import Pipeline

string_indexer = StringIndexer(inputCol='make', outputCol='make_indexed')

onehotencoder = OneHotEncoder(dropLast=False, inputCol= string_indexer.getOutputCol(), outputCol='make_encoded')

columns_for_model = [element[0] for element in cars.dtypes if element[1] != 'string' and 'symboling' not in element[0]] + [onehotencoder.getOutputCol()]
vector_assembler = VectorAssembler(inputCols=columns_for_model, outputCol='assembled_features')

rf = RandomForestClassifier(featuresCol=vector_assembler.getOutputCol(), labelCol='symboling_binary')

In [ ]:
# Respuesta

# Until here is the same as without pipeline but without doing fit / transform
# We use this objects as stages for our pipeline

pipeline = Pipeline(stages=[string_indexer, onehotencoder, vector_assembler, rf])

cars_pipeline_train, cars_pipeline_test = cars.randomSplit([0.8,0.2])

# We do fit and transform on the pipeline object

pipeline_model = pipeline.fit(cars_pipeline_train)

cars_pipeline = pipeline_model.transform(cars_pipeline_test)

cars_pipeline.show()

 

Los pipelines nos permiten reproducir todo el flujo cada vez que tenemos nuevos datos, de esta manera nos aseguramos de que cada nuevo "batch" se somete exactamente al mismo procesado.

**Para guardar el pipeline completo:**

In [ ]:
# Respuesta

pipeline_name = "random_forest_pipeline"
models_path = DATA_PATH + "models" + "_initials/" # change the last part using your initials
pipeline_model.save(models_path + pipeline_name)

 

**Podemos despues cargar el pipeline de la siguiente manera**

In [ ]:
# Respuesta

from pyspark.ml import PipelineModel

In [ ]:
# Respuesta

loaded_pipelinemodel = PipelineModel.load(models_path + pipeline_name)



# Ejercicio 1

Dado el siguiente DataFrame:

In [ ]:
from pyspark.sql.functions import col as c
from pyspark.sql import functions as F
from pyspark.ml.feature import StandardScaler, MinMaxScaler
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer, StringIndexerModel
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import RandomForestClassifier

df = spark.read.csv(DATA_PATH + 'data/pokemon.csv', header=True, inferSchema=True)



1) Realiza las transformaciones necesarias para comprobar que los datos son válidos para procesar (nombres de columnas, valores nulos).

2) La variable que se va a predecir es una derivada de la columna `Speed`; créala de forma que sea binaria, con 1 si el elemento supera la velocidad media del dataset, y 0 si no la supera.

3) Define una función que construya un `pipeline` con los pasos necesarios para preparar los datos y que aplique dicho pipeline a un DataFrame que reciba como parámetro de la función.

4) Construye un modelo de clasificación usando `pyspark.ml.classification.RandomForestClassifier` para predecir si la velocidad de un pokemon es mayor a la media.

5) Extrae la probabilidad de que la predicción sea 1 en una columna separada.

6) Construye una función que calcule los valores de *precision* y *recall* para diferentes valores de umbral (*threshold*), utilizando la definición (fórmulas) de cada métrica. Es decir, calcula el valor de las métricas para valores de umbral entre 0 y 1.

**Extra**: Dibuja las curvas de *precision* y *recall* versus el umbral (eje x).



**Parte 1**

In [ ]:
# Respuesta

for col in df.columns:
    df = df.withColumnRenamed(col, col.lower().replace('.', '').replace(' ', '_'))

In [ ]:
# Respuesta

print(df.count())
df = df.dropna(how='any')
print(df.count())



**Parte 2**

In [ ]:
# Respuesta

target = 'target'

avg = df.agg(F.avg('speed').alias('speed')).first()['speed']

df = df.withColumn('target', (c('speed') > avg).cast('double'))  # Both lines do the same
df = df.withColumn('target', F.when(c('speed') > avg, F.lit(1.0)).otherwise(F.lit(0.0)))    # Both lines do the same

df = df.drop('speed', 'sp_atk', 'sp_def', 'type_2')

In [ ]:
# Respuesta

df.show(2)



**Parte 3**

In [ ]:
# Respuesta

def preprocess_to_model(df, targetCol, path, subset=None, scaling=None, mode=True):
    if scaling not in (None, 'MinMax', 'Standard'):
        raise ValueError("Scaling value error. Valid values are: None, MinMax, Standard.")
    
    if subset is None:
        subset = [col for col in df.columns if col != targetCol]
    
    df = df.select(subset + [targetCol])
    
    categorical_cols = [col for col, type_ in df.dtypes 
                        if type_ == 'string'
                        and col != targetCol]
    boolean_cols = [col for col, type_ in df.dtypes 
                    if type_ == 'boolean'
                    and col != targetCol]
    numerical_cols = [col for col in df.columns 
                      if col not in categorical_cols
                      and col not in boolean_cols
                      and col != targetCol]
    
    str_idxs = [StringIndexer(inputCol=col, outputCol=col+'_idx', handleInvalid='skip')
                for col in categorical_cols] # handleInvalid='keep'
    
    ohes = [OneHotEncoder(inputCol=col+'_idx', outputCol=col+'_ohe')
            for col in categorical_cols]
    
    if scaling is not None:
        pre_scaling_assembler = VectorAssembler(inputCols=numerical_cols, 
                                                outputCol='numerical_features')
        
        if scaling == 'Standard':
            scaler = StandardScaler(withMean=True, withStd=True, 
                                    inputCol='numerical_features', 
                                    outputCol='numerical_features_scaled')
        else:
            scaler = MinMaxScaler(inputCol='numerical_features', 
                                    outputCol='numerical_features_scaled')
            
        assembler = VectorAssembler(inputCols=[col+'_ohe' for col in categorical_cols] + \
                                              ['numerical_features_scaled'] + \
                                              boolean_cols, 
                                outputCol='features')
        
        stages = str_idxs+ohes+[pre_scaling_assembler,scaler, assembler]
        
    else:
        assembler = VectorAssembler(inputCols=[col+'_ohe' for col in categorical_cols] + \
                                              numerical_cols + boolean_cols, 
                                outputCol='features')
        
        stages = str_idxs+ohes+[assembler]
    
    if mode:
        pipeline = Pipeline(stages=stages).fit(df)
        pipeline.save(path)
        print('Saving pipeline object on ' + path)
    else:
        pipeline = PipelineModel.load(path)
        print('Loading pipeline object from ' + path)
    
    df = pipeline.transform(df)
    
    return df.select('features', targetCol)

In [ ]:
# Respuesta

df_train, df_test = df.randomSplit([0.8, 0.2])

In [ ]:
# Respuesta

subset = [col for col in df_train.columns if col != 'name']

path = DATA_PATH + 'models' + '_initials/' + '/preprocess_to_model/' # change here "_initials" for your own initials

df_train_preprocesed = preprocess_to_model(df_train, target, path, subset=subset,
                                           scaling=None, mode=True)

df_test_preprocesed = preprocess_to_model(df_test, target, path, subset=subset,
                                          scaling=None, mode=False)

df_train_preprocesed.cache()
df_test_preprocesed.cache()



**Parte 4**

In [ ]:
# Respuesta

forest_classifier = RandomForestClassifier(featuresCol='features', labelCol=target)
forest_classifier = forest_classifier.fit(df_train_preprocesed)
df_forest_predicted = forest_classifier.transform(df_test_preprocesed)

df_forest_predicted.show(5)



**Parte 5**

In [ ]:
# Respuesta

from pyspark.sql.types import ArrayType, DoubleType

vector_to_array_udf = F.udf(lambda x: x.toArray().tolist(), ArrayType(DoubleType()))

df_forest_predicted = df_forest_predicted.withColumn('probability',
                                                         vector_to_array_udf(c('probability')))
df_forest_predicted = df_forest_predicted.withColumn('probability',  c('probability')[1])
df_forest_predicted.show(3)



**Parte 6**

In [ ]:
# Respuesta

import numpy as np

def calculate_metrics_by_threshold(df, pred_col, target, n_thresholds=99):
    df = df.select(pred_col, target)
    
    # explode by threshold
    thresholds = [np.round(x, 2) for x in np.linspace(0.01, 0.99, n_thresholds)]
    
    df = df.withColumn('threshold', F.array([F.lit(threshold) for threshold in thresholds]))
    df = df.withColumn('threshold', F.explode(c('threshold'))) # create one row per each threshold associated with each user (n_thresholds times the same user)
    
    # calculating metrics
    df = df.withColumn('PP', c(pred_col) > c('threshold')) ## PP = Predicted Positive 
    df = df.withColumn('TP', c('PP') & c(target).cast('boolean')) ## TP = True Positives 
    
    agg_df = df.groupBy('threshold').agg(F.sum(c('PP').cast('int')).alias('PP'),
                                         F.sum(c('TP').cast('int')).alias('TP'),
                                         F.sum(c(target).cast('int')).alias('RP'))
    agg_df = agg_df.withColumn('precision', c('TP')/c('PP'))
    agg_df = agg_df.withColumn('recall', c('TP')/c('RP'))
    agg_df = agg_df.withColumn('f1_score', 2 * (c('precision') * c('recall')) / (c('precision') + c('recall')))

    agg_df = agg_df.select('threshold', 'precision', 'recall', 'f1_score')
        
    return agg_df

In [ ]:
# Respuesta

df.select(c(target).cast('boolean'))

In [ ]:
# Respuesta

by_threshold = calculate_metrics_by_threshold(df_forest_predicted, 
                                              'probability', target, n_thresholds=99)
by_threshold

In [ ]:
# Respuesta

by_threshold = by_threshold.orderBy('threshold').toPandas()
by_threshold

In [ ]:
# Respuesta

from matplotlib import pyplot as plt

%matplotlib inline

by_threshold.plot(x='threshold', y=['precision', 'recall', 'f1_score'], figsize=(15,7))
plt.show()